# Лабораторная работа 6

**Внимание!** Данную лабораторную нужно выполнять на **Python 3.6+**.

## 1. Работа с метаклассами

### Задание 1.1 (0.6 балла)

Реализуйте метакласс **BoundedMeta**, который контролирует количество созданных объектов классов, которые имеют данный метакласс. Допустимое количество объектов задайте параметром (по умолчанию 1).

В случае превышения бросайте исключение **TypeError**. Eсли значение параметра - **None**, то ограничений нету.

Другими словами, у класса **C** с метаклассом **BoundedMeta** должно быть создано не более 2 экземпляров. 

In [ ]:
class C(metaclass=BoundedMeta, max_instance_count=2):
    pass

c1 = C()
c2 = C()

try:
    c3 = C()
except TypeError:
    print('everything works fine!')
else:
    print('something goes wrong!')

### Задание 1.2 (0.6 балла)

Реализуйте класс **BoundedBase**, в котором определен абстрактный classmethod get_max_instance_count, возвращающий максимальное количество экзмепляров, которые можно создать.

Не допускайте ***создания*** объекта, если данное значение превышено - бросайте исключение **TypeError**. Значение, равное **None** - без ограничений.

In [ ]:
class D(BoundedBase):
    @classmethod
    def get_max_instance_count(cls):
        return 1
    
d1 = D()

try:
    d2 = D()
except TypeError:
    print('everything works fine!')
else:
    print('something goes wrong!')

## 2. Работа с NumPy и SciPy

В области машинного обучения одним из самых популярных методов бинарной классификации (предсказываем один из двух классов, $+1$ или $-1$ для каждого объекта) является логистическая регрессия. Она выводится из метода максимального правдоподобия, который приводит к следующей задаче оптимизации:

$$ L(w, X, y) = \frac1{N}\sum_{i = 0}^{N} log (1 + exp(-y_ix_i^Tw)) + \frac{C}{2} ||w||^2 \longrightarrow \min_w$$
$$X \in R^{N \times M}, x \in R^{M}, w \in R^{M}, y \in \{-1, 1\}^N$$

Здесь $X$ - матрица объекты-признаки для обучающей выборки (по строкам объекты, по столбцам признаки), а $y$ - вектор ответов. Коэффициент $C$, вообще говоря, нужно подбирать отдельно, поскольку разные его значения приводят к разным решениям задачи оптимизации. Но в этой задаче положим $\mathbf{C = 10^{-3}}$

Когда мы решили задачу оптимизации (нашли $w$), мы принимаем решение о том, к какому классу относится объект по правилу $y(x) = sign(x^Tw)$.

In [ ]:
import numpy as np

Для тестирования правильности вычисления сгенерируем аргументы небольшого размера

In [ ]:
sample_size, features_count = 25, 10
w = np.random.random(features_count)
X, y = np.random.random((sample_size, features_count)), 2 * (np.random.randint(0, 2, sample_size) - 0.5)

### Задание 2.1 (0.2 балла)

In [ ]:
XY = np.hstack((X, y[:, np.newaxis]))
del X, y

Немного поработаем с ndarray. Получите из массива XY обратно X и y.

In [ ]:
# YOUR CODE HERE

### Задание 2.2 (0.2 балла)

Запрограммируйте вычисление функции L, используйте только матричные операции (внутри не должно быть циклов).

**Замечание**: Нигде в промежуточных вычислениях не стоит вычислять значение $\exp(−y_ix^Tw)$, иначе может произойти переполнение. Вместо этого следует напрямую вычислять необходимые величины с помощью специализированных для этого функций: `np.logaddexp` для `ln(1 + exp(·))` и `sp.special.expit` для `1/(1 + exp(-(·)))`.

In [ ]:
def logistic(w, X, y):
    """
        logistic(w, X, y) вычисляет функцию качества лог регрессии L(w, X, y)
        
        w: np.array размера (M,)
        X: np.array размера (N, M)
        y: np.array размера (M,)
        
        funcw: np.float 
    """
    # YOUR CODE HERE
    raise NotImplementedError()

### Задание 2.3.1 (0.3 балла)

Найдите градиент функции $\nabla_w L(w, X, y)$, запишите в терминах матричных операций.

Эффективно запрограммируйте вычисление градиента (опять же, только матричные операции!)

In [ ]:
def logistic_grad(w, X, y):
    '''
        logistic_grad(w, X, y) вычисляет градиент функции качества лог регрессии dL(w, X, y)/dw
        
        w: np.array размера (M,)
        X: np.array размера (N, M)
        y: np.array размера (M,)
        
        gradw: np.array размера (M,)
    '''
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert(logistic_grad(w, X, y).shape == w.shape)

### Задание 2.3.2 (0.3 балла)

Очень часто при подсчёте градиента допускаются ошибки, проверьте правильность реализации подсчёта градиента с помощью конечных разностей. 

$$[\nabla f(x)]_i \approx \frac{f(x + \varepsilon \cdot e_i) - f(x)}{\varepsilon}~~~~$$

где $e_i = (0, ... , 0, 1, 0, ..., 0)$ - i-й базисный орт, $\epsilon \approx 10^{-8}$

In [ ]:
def max_error(a, b): 
    return np.max(np.abs(a - b))


def grad_finite_diff(func, x, eps=1e-8):
    """
        w: np.array размера (M,)
        func: скалярная функция от векторного аргумента w, func(w) =  число
        eps: np.float константа для проверки градиента
        
        dnum: np.array размера (M,), численно посчитанный градиент
    """
    x, fval, dnum = x.astype(np.float64), func(x), np.zeros_like(x)

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
mat_grad = logistic_grad(w, X, y)
num_grad = grad_finite_diff(lambda w: logistic(w, X, y), w)

err = max_error(mat_grad, num_grad)
print('err = ', err, 'ok' if err < 1e-6 else 'ошибка очень большая!')

### Задание 2.4.1 (0.4 балла)

Для некоторых задач оптимизации очень удобно использовать гессиан.

Вычислите гессиан для функции L, запишите ответ в терминах матричных операций. Эффективно запрограммируйте вычисление гессиана.

In [ ]:
def logistic_hess(w, X, y):
    """
        logistic_hess(w, X, y) вычисляет гессиан функции качества лог регрессии dL(w, X, y)/dw
        
        w: np.array размера (M,)
        X: np.array размера (N, M)
        y: np.array размера (M,)
        
        hessw: np.array размера (M, M)
    """
    #hessw = # Гессиан dL/dw_iw_j
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert(logistic_hess(w, X, y).shape == (w.shape[0], w.shape[0]))

### Задание 2.4.2 (0.4 балла)

Теперь проверим правильность реализации подсчёта гессиана.

Для гессиана проверка выглядит похожим образом

$$[\nabla^2 f(x)]_{ij} \approx \frac{f(x + \varepsilon \cdot e_i + \varepsilon \cdot e_j) -f(x + \varepsilon \cdot e_i) - f(x + \varepsilon \cdot e_j)+ f(x)}{\varepsilon^2}~~~~~~~~~~~~~~~~~~~~~$$

где $e_i = (0, ... , 0, 1, 0, ..., 0)$ - i-й базисный орт, $\varepsilon \approx 10^{-4}$

In [ ]:
def hess_finite_diff(func, w, eps=1e-4):
    '''
        w: np.array размера (M,)
        func: скалярная функция от векторного аргумента w, func(w) =  число
        eps: np.float константа для проверки градиента
        
        dnum: np.array размера (M,), численно посчитанный градиент
    '''
    w, fval, dnum = w.astype(np.float64), func(w).astype(np.float64), np.zeros((w.size, w.size), dtype=np.float64)
    #dnum = # Вычислите численный гессиан d func/dw_iw_j для всех i, j
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
mat_grad = logistic_hess(w, X, y)
num_grad = hess_finite_diff(lambda w: logistic(w, X, y), w)

err = max_error(mat_grad, num_grad)

print('err = ', err)
print('ok' if max_error(mat_grad, num_grad) < 1e-4 else 'ошибка очень большая!')